# Chess

In [70]:
import chess, random
import torch
import torch.nn.functional as F

board = chess.Board()

In [103]:
for m in board.legal_moves:
    print(m)

g1h3
g1f3
b1c3
b1a3
h2h3
g2g3
f2f3
e2e3
d2d3
c2c3
b2b3
a2a3
h2h4
g2g4
f2f4
e2e4
d2d4
c2c4
b2b4
a2a4


In [688]:
def make_move_dict():
    move_dict = {}
    num_moves = 7

    dir = [0, 1, -1]

    for dir_x in dir:
        for dir_y in dir:
            if dir_x == 0 and dir_y == 0:
                continue
            for n in range(1, num_moves+1):
                move_dict[(dir_x*n, dir_y*n)] = len(move_dict)

    # knight moves
    move_dict[(2, 1)] = len(move_dict)
    move_dict[(2, -1)] = len(move_dict)
    move_dict[(-2, 1)] = len(move_dict)
    move_dict[(-2, -1)] = len(move_dict)
    move_dict[(1, 2)] = len(move_dict)
    move_dict[(1, -2)] = len(move_dict)
    move_dict[(-1, 2)] = len(move_dict)
    move_dict[(-1, -2)] = len(move_dict)

    # promotions
    for piece in range(4):
        move_dict[(0, 1, piece)] = len(move_dict)
        move_dict[(1, 1, piece)] = len(move_dict)
        move_dict[(-1, 1, piece)] = len(move_dict)

    move_dict_inv = {v: k for k, v in move_dict.items()}

    return move_dict, move_dict_inv


class BoardLogic:

    def __init__(self):
        self.board = chess.Board()
        self.unique_pieces = sorted(set(str(board).replace("\n", "").replace(" ", "")))
        self.piece_to_idx = {p: i for i, p in enumerate(self.unique_pieces)}

        self.alpha_idx = {c: i for i, c in enumerate("abcdefgh")}
        self.numeric_idx = {c: i for i, c in enumerate("12345678")}
        self.promotion_idx = {c: i for i, c in enumerate("qrbn")}

        self.move_dict, self.move_dict_inv = make_move_dict()

    def board_to_state(self, board):
        state = [row.split(" ") for row in str(board).split("\n")]
        state = [[self.piece_to_idx[p] for p in row] for row in state]
        state_onehot = F.one_hot(torch.tensor(state), num_classes=len(self.unique_pieces)).float().permute(2, 0, 1)
        state_onehot = state_onehot[1:].unsqueeze(0) 
        return state_onehot

    def move_get_origin(self, move):
        move = str(move)
        x, y = self.alpha_idx[move[0]], self.numeric_idx[move[1]]
        return x + y*8

    def move_get_delta(self, move):
        move = str(move)
        x1, y1 = self.alpha_idx[move[0]], self.numeric_idx[move[1]]
        x2, y2 = self.alpha_idx[move[2]], self.numeric_idx[move[3]]
        delta = (x2-x1, y2-y1)

        if len(move) == 5:  # promotion
            piece = self.promotion_idx[move[4].lower()]
            delta = (*delta , piece)

        return self.move_dict[delta]

        
    def move_to_action(self, move):
        origin = self.move_get_origin(move)
        delta = self.move_get_delta(move)

        return origin * len(self.move_dict) + delta

    def action_to_move(self, action):
        origin, delta = action//len(self.move_dict), action%len(self.move_dict)
        x = origin % 8
        y = origin // 8

        move_delta = self.move_dict_inv[delta]
        move = "abcdefgh"[x] + str(y+1) + "abcdefgh"[x + move_delta[0]] + str(y + 1 + move_delta[1])
        if len(move_delta) == 3:  # promotion
            move += "qrbn"[move_delta[2]]

        return chess.Move.from_uci(move)


In [691]:
bts = BoardLogic()
board = chess.Board()
mirror = False
check_mate = False
counter = 0

In [692]:
while not check_mate:
    counter += 1
    for m in board.legal_moves:
        board.push(m)
        if board.is_checkmate():
            print("board.is_checkmate()")
            check_mate = True
        board.pop()

    moves = torch.tensor([bts.move_to_action(m) for m in board.legal_moves], dtype=torch.long)
    Q = torch.randn(64 * 76)
    action = int(moves[torch.argmax(Q[moves])])
    move = bts.action_to_move(action)
    board.push(move)

    board = board.mirror()
    mirror = not mirror

print(counter)

KeyboardInterrupt: 

In [686]:
for m in board.legal_moves:
    board.push(m)
    if board.is_checkmate():
        print("board.is_checkmate()")
        check_mate = True
        if mirror:
            print(board.mirror())
        else:
            print(board)
    board.pop()

moves = torch.tensor([bts.move_to_action(m) for m in board.legal_moves], dtype=torch.long)
print(len(moves))
Q = torch.randn(64 * 76)
action = int(moves[torch.argmax(Q[moves])])
print(action)
print(action//76, action%76)
move = bts.action_to_move(action)
board.push(move)

board = board.mirror()
mirror = not mirror

if mirror:
    print(board.mirror())
else:
    print(board)

board.is_checkmate()
. . b q k b r .
. p p . . Q p n
r . n p P . . .
p . . . . . P p
N . . P P . . .
. P . . B . . P
P . P . K . B .
R . . . . . N R
41
926
12 14
. . b q k b r .
. p p . . p p n
r . n p P . . .
p . . . . . P p
N . . P P . . .
. P . . B . . P
P . P . . K B .
R . . . . Q N R
